Modied from: https://github.com/sonyc-project/urban-sound-tagging-baseline

In [ ]:
import argparse
import gzip
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

import tensorflow as tf

from vggish import vggish_input
from vggish import vggish_postprocess
from vggish import vggish_slim

In [ ]:
dataset_dir = r"D:\DCASE_2019\audio\train" # where the audio files are
output_dir = r"D:\DCASE_2019\audio\features_train" # where to save vgg embeddigns
vggish_resource_dir = r"D:\DCASE_2019\baseline\urban-sound-tagging-baseline\urban-sound-tagging-baseline\vggish" # where the vgg checkpoint file is

In [ ]:
def make_extract_vggish_embedding(frame_duration=.96, hop_duration=.96, 
                                  input_op_name='vggish/input_features',
                                  output_op_name='vggish/embedding', 
                                  embedding_size=128, resources_dir=None):
    """
    Creates a coroutine generator for extracting and saving VGGish embeddings

    Parameters
    ----------
    frame_duration
    hop_duration
    input_op_name
    output_op_name
    embedding_size
    resources_dir

    Returns
    -------
    coroutine

    """
    params = {
        'frame_win_sec': frame_duration,
        'frame_hop_sec': hop_duration,
        'embedding_size': embedding_size
    }
    print(params)
    if not resources_dir:
        resources_dir = os.path.join(os.path.dirname(__file__), 'vggish/resources')

    pca_params_path = os.path.join(resources_dir, 'vggish_pca_params.npz')
    model_path = os.path.join(resources_dir, 'vggish_model.ckpt')

    try:
        with tf.Graph().as_default(), tf.Session() as sess:
            # Define the model in inference mode, load the checkpoint, and
            # locate input and output tensors.
            net = vggish_slim.define_vggish_slim(training=False, **params)
            vggish_slim.load_vggish_slim_checkpoint(sess, model_path, **params)
            print(dir(net))
            quit()
            while True:
                # We use a coroutine to more easily keep open the Tensorflow contexts
                # without having to constantly reload the model
                audio_path, output_path = (yield)

                if os.path.exists(output_path):
                    continue

                try:
                    examples_batch = vggish_input.wavfile_to_examples(audio_path, **params)
                except ValueError:
                    print("Error opening {}. Skipping...".format(audio_path))
                    continue

                # Prepare a postprocessor to munge the model embeddings.
                pproc = vggish_postprocess.Postprocessor(pca_params_path, **params)

                input_tensor_name = input_op_name + ':0'
                output_tensor_name = output_op_name + ':0'

                features_tensor = sess.graph.get_tensor_by_name(input_tensor_name)
                embedding_tensor = sess.graph.get_tensor_by_name(output_tensor_name)

                # Run inference and postprocessing.
                [embedding_batch] = sess.run([embedding_tensor],
                                             feed_dict={features_tensor: examples_batch})

                emb = pproc.postprocess(embedding_batch, **params).astype(np.float32)

                with gzip.open(output_path, 'wb') as f:
                    emb.dump(f)

    except GeneratorExit:
        print('pass')
        pass

In [ ]:
def extract_embeddings_vggish(dataset_dir, output_dir,
                              vggish_resource_dir, frame_duration=0.96,
                              hop_duration=0.96, progress=True,
                              vggish_embedding_size=128):
    """
    Extract embeddings for files annotated in the SONYC annotation file and save them to disk.

    Parameters
    ----------
    dataset_dir
    output_dir
    vggish_resource_dir
    frame_duration
    hop_duration
    progress
    vggish_embedding_size

    Returns
    -------

    """

    print("* Loading annotations.")
#     annotation_data = pd.read_csv(annotation_path).sort_values('audio_filename')

    extract_vggish_embedding = make_extract_vggish_embedding(frame_duration, hop_duration,
        input_op_name='vggish/input_features', output_op_name='vggish/embedding',
        resources_dir=vggish_resource_dir, embedding_size=vggish_embedding_size)
    # Start coroutine
    next(extract_vggish_embedding)

    out_dir = os.path.join(output_dir, 'vggish')
    os.makedirs(out_dir, exist_ok=True)


    print("* Extracting embeddings.")
    for file in os.listdir(dataset_dir):
        audio_path = os.path.join(dataset_dir, file)
        filename, _ = os.path.splitext(file)
        emb_path = os.path.join(out_dir, filename + '.npy.gz')
        extract_vggish_embedding.send((audio_path, emb_path))

    extract_vggish_embedding.close()

In [ ]:
extract_embeddings_vggish(dataset_dir, output_dir, vggish_resource_dir)